In [ ]:
import torch
import torch.nn as nn

class RevIN(nn.Module):
    def __init__(self, num_features: int, p_lag:int, eps=1e-5, affine=True):
        super(RevIN, self).__init__()
        self.p_lag = p_lag
        self.num_features = num_features
        self.eps = eps
        self.affine = affine
        if self.affine:
            self._init_params()

    def forward(self, x, mode:str):
        if mode == 'norm':
            self._get_statistics(x)
            x = self._normalize(x)
        elif mode == 'denorm':
            x = self._denormalize(x)
        else: raise NotImplementedError
        return x

    def _init_params(self):
        self.affine_weight = nn.Parameter(torch.ones(self.num_features, self.p_lag))
        self.affine_bias = nn.Parameter(torch.zeros(self.num_features, self.p_lag))

    def _get_statistics(self, x):
        self.mean = torch.mean(x, dim=2, keepdim=True).detach()
        self.stdev = torch.sqrt(torch.var(x, dim=2, keepdim=True, unbiased=False) + self.eps).detach()

    def _normalize(self, x):
        x = x - self.mean
        x = x / self.stdev
        if self.affine:
            x = x * self.affine_weight
            x = x + self.affine_bias
        return x

    def _denormalize(self, x):
        if self.affine:
            x = x - self.affine_bias
            x = x / (self.affine_weight + self.eps*self.eps)
        x = x * self.stdev
        x = x + self.mean
        return x
    
class RLinear(nn.Module):
    def __init__(self, p_lag, n_features, future_steps, batch_size = 8):
        super(RLinear, self).__init__()
        self.linear_layer = nn.Linear(p_lag * n_features, p_lag * 1)
        self.revin_layer = RevIN(num_features = n_features, p_lag = p_lag)
        self.criterion = nn.MSELoss()   
        self.p_lag = p_lag
        self.batch_size = batch_size
        self.n_features = n_features
        self.future_steps = future_steps

    def forward(self, input):
        input = input.float()
        input_normalized = self.revin_layer(input.reshape(self.batch_size,self.n_features,self.p_lag), 'norm')
        input_transformed = self.linear_layer(input_normalized.reshape(self.batch_size, self.p_lag*self.n_features))
        input_denormalized = self.revin_layer(input_transformed.reshape(self.batch_size,1,self.p_lag), 'denorm')  
        return input_denormalized


training_df, val_df, test_df = split_dataset(ETTm2)
train_data = DataLoader(TimeSeriesDataset(training_df, future_steps= 7, target_column = ['OT'], p_lag=10), batch_size=2, drop_last=True)
ri = RevIN(7,10, affine=True)
rlinear = RLinear(10,7,7,2)
for idx, (i,t) in enumerate(train_data): 
    #peter = ri(i.reshape(2,7,10), 'norm')
    #print(peter)
    #repeter = ri(peter.reshape(2,7,10), 'denorm')
    #print(repeter)
    print(rlinear(i))
    if idx == 1: 
        break